# Automação Web

## Desafio

Trabalhamos em ima importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro
Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.


Para isso, vamos criar uma automação web:
- Usaremos o Selenium
    - **Importante:** Baixar o Webdrive

### Objetivos
- Capturar a cotação do Dólar
- Capturar a cotação do Euro
- Capturar a cotação do Ouro
- Importar a base de dados
- Atualizar a cotação, o preço de compra e o preço de vendas
- Exportar o relatório atualizado


## Preparando o Sistema
- Baixar o Webdriver
    - Google Chrome -> chromedriver
    - Mozila Firefox -> geckodriver
    - colocar na mesma pasta do código fonte
- Importar bibliotecas
- Criar o navegador

In [34]:
#Importando o Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Criando navegador
navegador = webdriver.Chrome('chromedriver')

### Cotação do Dólar

In [35]:
#Acessando o site do Google e acessando xpath
#Cotação Dólar
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_dolar)

5.510841999999999


### Cotação do Euro

In [36]:
#Cotação Euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_euro)

6.3669513040000005


### Cotação do Ouro

In [37]:
#Cotação Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value').replace(',','.')
print(cot_ouro)

#Fechando o navegador
navegador.quit()

310.76


## Atualizando a base de dados

In [38]:
import pandas as pd

df_produtos = pd.read_excel('datas/3_produtos.xlsx')
display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [39]:
#Atualizando a Cotação
df_produtos.loc[df_produtos['Moeda'] == 'Dólar' , 'Cotação'] = float(cot_dolar)
df_produtos.loc[df_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cot_euro)
df_produtos.loc[df_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cot_ouro)

display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.510842,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.366951,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.510842,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.510842,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.366951,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.510842,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,310.760000,7000.00,1.15,8050.000


In [42]:
#Atualizar o preço de compra
df_produtos['Preço Base Reais'] = df_produtos['Preço Base Original'] * df_produtos['Cotação']

#Atualizar preço de vendas
df_produtos['Preço Final'] = df_produtos['Preço Base Reais'] * df_produtos['Margem']

#Formatando o preço de venda
#df_produtos['Preço Final'] = df_produtos['Preço Final'].map('R${:.2f}'.format)

display(df_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.510842,5510.786892,1.40,7715.101648
1,Carro Renault,4500.00,Euro,6.366951,28651.280868,2.00,57302.561736
2,Notebook Dell,899.99,Dólar,5.510842,4959.702692,1.70,8431.494576
3,IPhone,799.00,Dólar,5.510842,4403.162758,1.70,7485.376689
4,Carro Fiat,3000.00,Euro,6.366951,19100.853912,1.90,36291.622433
5,Celular Xiaomi,480.48,Dólar,5.510842,2647.849364,2.00,5295.698728
6,Joia 20g,20.00,Ouro,310.760000,6215.200000,1.15,7147.480000


In [48]:
#Exportando a tabela
from datetime import datetime
data = datetime.today().strftime('%Y-%m-%d_%H-%M')
df_produtos.to_excel(f'datas/3_produtos_{data}.xlsx', index=False)